In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt

from lblm_inference import LBLMInferenceEngine

ckpt = "ckpt_epoch358.pt"
device = "mps"
eng = LBLMInferenceEngine(ckpt, device=device)

Package 'model2' initialized.


/Users/adamsobieszek/PycharmProjects/NeuroLM-Decode/lblm_inference.py:91: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  chk = torch.load(ckpt_path, map_location=device)


[load] Model ready (14,946,560 params) – device=mps dtype=torch.float32


In [ ]:
from lblm_dataset import EEGPickleDataset
import glob
import os
dataset_dir="Protocol_v8_pkl_3word"

train_pickle_files = glob.glob(os.path.join(dataset_dir, "**", "*.pkl"), recursive=True)
train_pickle_files = [f for f in train_pickle_files]

dataset_train = EEGPickleDataset(
    filepaths=train_pickle_files,
    expected_channels=62,
    expected_sfreq=200,
    model_patch_length=25,
    map_filename_to_subject_id=True, 
    default_subject_id=0,
    desired_processing_duration_sec=3,
    apply_multi_band_mixing=True,
    iterate_all_bands_per_file=True,
    # fixed_range =(200,800)
)
print(f"Training dataset size: {len(dataset_train)}")

from torch.utils.data import DataLoader
data_loader_train = DataLoader(
    dataset_train,
    batch_size=1,
    shuffle=False,
    num_workers=1,
    pin_memory=True,
)

Multi-band processing enabled. Defined bands: {'raw': (1.0, 50.0), 'alpha': (8.0, 13.0), 'beta': (13.0, 30.0), 'gamma': (30.0, 50.0)}
Mode: Iterating through all 4 bands for each of the 2568 files.
EEGPickleDataset initialized with 2568 original files.
Effective dataset size: 5136 samples.
Dataset assumes data is at 200 Hz.
Training dataset size: 5136
tensor([0], device='mps:0')


tensor(1)

In [ ]:
batch = next(iter(data_loader_train))

x_eeg = batch['x_raw_eeg'].to(device, non_blocking=True)
subject_ids = batch['subject_id'].to(device, non_blocking=True)
targets = batch['target'].to(device, non_blocking=True)
print(subject_ids)


preds = eng.predict(x_eeg, subject_id=subject_ids, mask_ratio=0., targets=targets)
lentent, class_pred = preds
ensemble_prediction = class_pred.cpu().mode().values
ensemble_prediction